In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-229836
Azure region: eastus2
Subscription id: 6971f5ac-8af1-446e-8034-05acea24681f
Resource group: aml-quickstarts-229836


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute

cluster_name = 'vm-proj'

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.


try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
except:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', 
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True)

InProgress.
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [5]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os

# Specify parameter sampler
ps = RandomParameterSampling({"C":  uniform(0, 1), "max_iter": choice(25, 50, 100, 150, 200, 250)})

# Specify a Policy
policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)

if "training" not in os.listdir():
    os.mkdir("./training")

# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

# Create a ScriptRunConfig Object to specify the configuration details of your training job
src = ScriptRunConfig('.', 'train.py', compute_target=compute_target, environment=sklearn_env)

# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(run_config=src,
                                     hyperparameter_sampling=ps,
                                     policy=policy,
                                     primary_metric_name='Accuracy',
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                     max_total_runs=20,
                                     max_concurrent_runs=4)

In [6]:
# Submit your hyperdrive run to the experiment and show run details with the widget.
hyperdrive_run = exp.submit(config=hyperdrive_config)
RunDetails(hyperdrive_run).show()
hyperdrive_run.wait_for_completion(show_output=True)

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_16609810-7164-44b2-a618-0311bb336eec
Web View: https://ml.azure.com/runs/HD_16609810-7164-44b2-a618-0311bb336eec?wsid=/subscriptions/6971f5ac-8af1-446e-8034-05acea24681f/resourcegroups/aml-quickstarts-229836/workspaces/quick-starts-ws-229836&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

[2023-04-05T15:58:18.909116][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space
[2023-04-05T15:58:19.3604433Z][SCHEDULER][INFO]Scheduling job, id='HD_16609810-7164-44b2-a618-0311bb336eec_0' 
[2023-04-05T15:58:19.4683063Z][SCHEDULER][INFO]Scheduling job, id='HD_16609810-7164-44b2-a618-0311bb336eec_1' 
[2023-04-05T15:58:19.606260][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.
[2023-04-05T15:58:19.6280648Z][SCHEDULER][INFO]Scheduling job, id='HD_16609810-7164-44b2-a618-0311bb336eec_2' 
[2023-04-05T15:58:19.6922754Z][SCHEDULER][INFO]Scheduling job, id='HD_16609810-7164-44b2-a618-0311bb336e

{'runId': 'HD_16609810-7164-44b2-a618-0311bb336eec',
 'target': 'vm-proj',
 'status': 'Completed',
 'startTimeUtc': '2023-04-05T15:58:18.088972Z',
 'endTimeUtc': '2023-04-05T16:15:19.988242Z',
 'services': {},
 'properties': {'primary_metric_config': '{"name":"Accuracy","goal":"maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': '9b2c463a-02f3-4d87-968b-d35fd25b9db4',
  'user_agent': 'python/3.8.10 (Linux-5.15.0-1031-azure-x86_64-with-glibc2.17) msrest/0.7.1 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.48.0',
  'space_size': 'infinite_space_size',
  'score': '0.9088012139605463',
  'best_child_run_id': 'HD_16609810-7164-44b2-a618-0311bb336eec_16',
  'best_metric_status': 'Succeeded',
  'best_data_container_id': 'dcid.HD_16609810-7164-44b2-a618-0311bb336eec_16'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'configuration': None,
  'attribution': None,
  'telemetryValue

In [20]:
import joblib
# Get your best run and save the model from that run.

best_run = hyperdrive_run.get_best_run_by_primary_metric()
print(best_run.get_details())


{'runId': 'HD_16609810-7164-44b2-a618-0311bb336eec_16', 'target': 'vm-proj', 'status': 'Completed', 'startTimeUtc': '2023-04-05T16:13:59.594672Z', 'endTimeUtc': '2023-04-05T16:14:24.836553Z', 'services': {}, 'properties': {'_azureml.ComputeTargetType': 'amlctrain', 'ContentSnapshotId': '9b2c463a-02f3-4d87-968b-d35fd25b9db4', 'ProcessInfoFile': 'azureml-logs/process_info.json', 'ProcessStatusFile': 'azureml-logs/process_status.json'}, 'inputDatasets': [], 'outputDatasets': [], 'runDefinition': {'script': 'train.py', 'command': '', 'useAbsolutePath': False, 'arguments': ['--C', '0.35978778340791295', '--max_iter', '100'], 'sourceDirectoryDataStore': None, 'framework': 'Python', 'communicator': 'None', 'target': 'vm-proj', 'dataReferences': {}, 'data': {}, 'outputData': {}, 'datacaches': [], 'jobName': None, 'maxRunDurationSeconds': 2592000, 'nodeCount': 1, 'instanceTypes': [], 'priority': None, 'credentialPassthrough': False, 'identity': None, 'environment': {'name': 'sklearn-env', 'vers

In [ ]:

model = best_run.register_model(model_name='best_reg_bank')
filename = "best_logreg_bank.pkl"
joblib.dump(value=model, filename=filename)

In [ ]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

ds = TabularDatasetFactory.from_delimited_files(
    path="https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv", 
)

In [ ]:
from train import clean_data
from sklearn.model_selection import train_test_split

# Use the clean_data function to clean your data.

x, y = clean_data(ds)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [ ]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='AUC_weighted',
    training_data=x_train,
    label_column_name='y',
    n_cross_validations=5)

In [ ]:
# Submit your automl run

run = exp.submit(config=automl_config, show_output=True)

In [ ]:
# Retrieve and save your best automl model.

best_run = run.get_best_child()
model_name = best_run.properties["model_name"]
model = run.register_model(model_name=model_name)
filename = 'best_automl_bank.pkl'
joblib.dump(value=model, filename=filename)